In [254]:
from causallearn.search.ConstraintBased.PC import pc

### Changing sample size
np.random.seed(1)
d=6  #Dimensionality
supp=(1,2) #support indices
s=len(supp)  #Number of support entries
sample=[8,10,12,14,16,18,20]
fn=np.zeros((len(sample)))
fp=np.zeros((len(sample)))
fnpc=np.zeros((len(sample)))
fp=np.zeros((len(sample)))
runs=100
l=1 # Number of intervals we combine for the statistics
B=100 # Bootstrap runs
I=50      #Number of Intervals 
a=[list(itertools.combinations(range(d), k)) for k in range(0,d+1)]
subsets = [item for sublist in a for item in sublist]
dic={}
# We first fix the data for all runs, to eliminate the randomness of this.
betas=np.zeros((runs,I,d))
for r in range(runs):
    for i in range(I):
        betas[r,i,supp]=np.random.uniform(low=1,high=5,size=(s))

for o,n in enumerate(sample):
    print('Sample size',o+1,'of',len(sample))
    for r in range(runs):
        X=[]
        Y=[]
        for i in range(I):
            x=np.zeros((n,d))
            y=np.zeros((n))
            std=np.random.uniform(low=1,high=5,size=(d))
            x[:,0]=np.random.normal(scale=std[0],size=(n))
            x[:,1]=x[:,0]+np.random.normal(scale=std[1],size=(n))
            x[:,2]=0.3*x[:,1]+np.random.normal(scale=std[2],size=(n))
            x[:,3]=0.2*x[:,2]+np.random.normal(scale=std[3],size=(n))
            e=np.random.multivariate_normal(0*np.ones(n),1*np.eye(n))
            y=x@betas[r,i,:]+e
            x[:,4]=np.random.normal(scale=std[4],size=(n))+0.1*x[:,1]+y
            x[:,5]=np.random.normal(scale=std[5],size=(n))+y
            x_pc[i*n:(i+1)*n,:6]=copy.copy(x)
            x_pc[i*n:(i+1)*n,:6]=copy.copy(y)
            X.append(x)
            Y.append(y)

            
        plausibleS=loli.gauss(X,Y)
        if not not plausibleS:
            supphat=set.intersection(*plausibleS)
            if len(supphat.difference(set(supp)))>0:
                fp[o]+=1/runs
            if len(set(supp).difference(supphat))>0:
                fn[o]+=1/runs
                
        cg = pc(x_pc,alpha=0.1)
                


fig, ax = plt.subplots()
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(14)
plt.plot(sample,0.1*np.ones((len(sample))),label='Confidence level',color='black')
plt.plot(sample,fp,label='False Positive SEM',linestyle=(0, (1, 5)))
plt.plot(sample,fn,label='False Negative SEM',linestyle=(0, (1, 1)))
plt.plot(sample,fpind,label='False Positive GAUSS',linestyle=(0, (5, 5)))
plt.plot(sample,fnind,label='False Negative GAUSS',linestyle=(0, (5, 1)))

plt.xlabel('Sample Size Per Environment',fontsize=14)
plt.ylabel('FP/FN Rate',fontsize=14)
plt.legend()
# plt.savefig('Figures/Samplechangen.eps',bbox_inches="tight",format='eps')





  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Graph Nodes:
X1;X2;X3

Graph Edges:
1. X1 o-o X2
2. X2 o-o X3

[[ 0 -1  0]
 [-1  0 -1]
 [ 0 -1  0]]


In [263]:
import numpy as np
from causallearn.search.FCMBased import lingam
supp=(1,2)
s=len(supp)
d=7
n=500 # Sample size
I=1      #Number of Intervals 
# We first fix the data for all runs, to eliminate the randomness of this.
betas=np.zeros((I,d))



x=np.zeros((I*n,d))
for i in range(I):
    betas[i,supp]=np.random.uniform(low=1,high=1,size=(s))
    std=np.random.uniform(low=1,high=1,size=(d))
    x[i*n:(i+1)*n,0]=np.random.uniform(low=-std[0],high=std[0],size=(n))
    x[i*n:(i+1)*n,1]=x[i*n:(i+1)*n,0]+np.random.uniform(low=-std[1],high=std[1],size=(n))
#     x[i*n:(i+1)*n,0]=np.random.normal(scale=10*std[0],size=(n))
#     x[i*n:(i+1)*n,1]=x[i*n:(i+1)*n,0]+np.random.normal(scale=std[1],size=(n))
    x[i*n:(i+1)*n,2]=0.3*x[i*n:(i+1)*n,1]+np.random.uniform(low=-std[1],high=std[1],size=(n))
    x[i*n:(i+1)*n,3]=0.2*x[i*n:(i+1)*n,2]+np.random.uniform(low=-std[1],high=std[1],size=(n))
    e=np.random.uniform(low=-std[1],high=std[1],size=(n))
#     e=np.random.uniform(low=-1,high=1,size=(n))
    x[i*n:(i+1)*n,6]=x[i*n:(i+1)*n,:]@betas[i,:]+e
    x[i*n:(i+1)*n,5]=np.random.uniform(low=-std[1],high=std[1],size=(n))
    x[i*n:(i+1)*n,4]=np.random.uniform(low=-std[1],high=std[1],size=(n))+0.1*x[i*n:(i+1)*n,1]+x[i*n:(i+1)*n,6]+x[i*n:(i+1)*n,5]
data=x
# default parameters
cg = pc(data,alpha=0.1)
G, edges = fci(data)

# print(G)
# print(cg.G.graph)

model = lingam.ICALiNGAM()
model.fit(data)
# print(model.causal_order_)
print(np.round(model.adjacency_matrix_))


  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

X7 --> X5
[[0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]]


The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsIC())

If you wish to pass a sample_weight